In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing
import tensorflow as tf


In [6]:
df = pd.read_csv(
    "datas/EURUSD_H1.csv",
    sep="\t",
    nrows=2**16,
    header=0,
    dtype={
        "open": np.float32,
        "close": np.float32,
        "tickvol": np.float32,
        "vol": np.float32,
    },
    parse_dates=[[0,1]],
    index_col=[0],
    names=["date","time","open","high","low","close","tickvol","vol","spread"])

In [23]:
prices = np.nan_to_num(np.array(df["open"]), posinf=0, neginf=0).reshape((-1,16,1))
prices_diff = np.diff(prices[: -64])
print(prices.shape)
norm = tf.keras.layers.LayerNormalization(axis=1)
nx=norm(prices).numpy()
np.min(nx)
# print(norm(prices).numpy())

(4096, 16, 1)


-0.7367172

In [44]:
print(len(df.index))
df = df.asfreq(freq='5min')
df = df.fillna(method='ffill')
print(len(df.index))

39314
54935


In [24]:
def shifted_mse(y_true, y_pred):
    y_true_sign = tf.math.softsign(y_true)
    y_pred_sign = tf.math.softsign(y_pred)
    d = tf.keras.losses.mean_squared_error(y_true + y_true_sign, y_pred + y_pred_sign)
    return d

def mse(y_true, y_pred):
    d = tf.keras.losses.mean_squared_error(y_true, y_pred)
    return d 


In [44]:
x=np.array([-1.0])
y=np.array([0.01])
print(shifted_mse(x,y).numpy())
print(mse(x,y).numpy())

2.3100990197039506
1.0201


In [28]:
norm = tf.keras.layers.LayerNormalization(axis=1)
x = np.arange(256, dtype='float32').reshape((16,16,1))
#print(x)
